In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
import json

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.keras.backend.set_floatx('float16')

In [5]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [6]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0,
    train_size=499
)
gpt.load_weights('./gpt_weight_pretrain/weight_fp16')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


gpt.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
)

float16 <dtype: 'float16'> True


In [7]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [8]:
ids = cbpe.encode('今天天气还行')
ids

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.570 seconds.
Prefix dict has been built successfully.


[837, 259, 497, 57, 8, 237]

In [9]:
def test_basic_logic():
    ids = cbpe.encode('今天天气还行')
    #print(ids)
    print("+" * 20)
    for i in range(10):
        output = gpt(tf.constant([ids]))
        #print(output[0].shape)
        nid = np.argmax(output[0, -1])
        ids += [nid]
        #print(i, cbpe.decode(ids))
        #print(np.argmax(output[0],axis=-1))
        print(cbpe.decode(np.argmax(output[0],axis=-1)))
        print('-' * 30)
test_basic_logic()

++++++++++++++++++++
气 不错 不错行 
------------------------------
气 不错 不错行 ,
------------------------------
气 不错 不错行 , 我
------------------------------
气 不错 不错行 , 我 就
------------------------------
气 不错 不错行 , 我 就 想
------------------------------
气 不错 不错行 , 我 就 想着
------------------------------
气 不错 不错行 , 我 就 想着 去
------------------------------
气 不错 不错行 , 我 就 想着 去 看看
------------------------------
气 不错 不错行 , 我 就 想着 去 看看 
------------------------------
气 不错 不错行 , 我 就 想着 去 看看 ,
------------------------------


In [10]:
def get_learning_rate(learning_rate=6e-4,
                      warmup_steps=20_0000,
                      decay_steps=200_0000,
                      alpha=0.0):
    def decayed_learning_rate(step=1):
        if step <= warmup_steps:
            mult = step / float(warmup_steps)
        else:
            progress = (step - warmup_steps) / (decay_steps - warmup_steps)
            mult = 0.5 * (1 + math.cos(math.pi * progress))
            mult = max(0.1, mult)
        return learning_rate * mult
    return decayed_learning_rate

In [11]:
train_recoard = json.load(open('./labeled_data/advanture_translated/truncated_advanture_train.json'))
valid_recoard = json.load(open('./labeled_data/advanture_translated/truncated_advanture_valid.json'))

In [12]:
len(train_recoard),len(valid_recoard)

(4500, 266)

In [13]:
import copy
def data_generator(stories, batch_size=4,sample_len=200,inf=False):
    while True:
        batch_data = []
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            story_content = one_story.to_dungeon_format()
            story_content = story_content.replace("<start>\n","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace(" ","")
            ids = cbpe.encode(story_content)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break
            
import copy
def data_generator_content(texts, batch_size=4,sample_len=200,inf=False):
    tmp_text = copy.copy(texts)
    segments = []
    while tmp_text:
        story_content = tmp_text[:10000]
        segments.append(story_content)
        tmp_text = tmp_text[10000:]
    while True:
        
        batch_data = []
        random.shuffle(segments)
        for one_story_context in segments:
            ids = cbpe.encode(one_story_context)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break
            
def turb_sentence(inputs):
    outputs = []
    for i in inputs:
        if random.random() > 0.1:
            outputs.append(i)
    return outputs
            
def data_generator_json(stories, batch_size=1,inf=False):
    while True:
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            previous_contents = one_story["previous"]
            previous_contents = turb_sentence(previous_contents)
            action = one_story["action"]
            content = one_story["result"]
            
            previous_contents = ''.join(previous_contents)
            content = ''.join(content)
            
            ids_prev = cbpe.encode(previous_contents + "\n")
            ids_action = cbpe.encode(">" + action + "\n")
            ids_content = cbpe.encode(content)[:300]
            
            ids_prev = ids_prev + ids_action
            
            prev_context_len = int(random.uniform(100,500))
            ids_prev = ids_prev[-prev_context_len:]
            
            overall_ids = ids_prev + ids_content
            mask = [0 for i in ids_prev] + [1 for i in ids_content]
            yield np.asarray([overall_ids]),np.asarray([mask])
            
        if not inf:
            break

In [14]:
valid_gen = data_generator_json(valid_recoard,inf=True)

In [15]:
x,m = valid_gen.__next__()

In [16]:
x.shape,m.shape

((1, 368), (1, 368))

In [17]:
cbpe.decode(x[0])

'打开 冰箱 , 迎接 你 的 是 一个 宏伟 的 霉菌 群 。 但是 , 可以 节省 三瓶 水 。 你 检查 厨房 你 穿过 橱柜 , 找到 几罐 豆子 。 你 把 它们 拉 出来 放在 柜台 上 。 你 听到 另 一个 人 在 楼上 翻找 东西 。 你 拿 钥匙 你 打开 房子 的 后门 , 进去 。 你 在 一个 厨房 和 客厅 相结合 的 地方 。 您 会 看到 几个 橱柜门 敞开 着 , 一台 冰箱 和 几张 沙发 围绕 着 电视 。 你 搜索 办公桌 区域 你 开始 扫描 桌子 , 有 很多 文件 和 收据 。 你 打开 第一个 抽屉 , 找到 一些 钥匙 。 你 开门 到 前台 区 你 慢慢 打开门 , 看到 前台 区域 几乎 没有 动过 , 周围 飞扬 着 很多 灰尘 。 你 爬 过 窗户 你 把 自己 拉 过去 , 发现自己 在 一个 小 浴室 里 。 这里 似乎 没有 什么 重要 的 事情 。 你 打破 窗户 你 检查 前台 你 检查 前门 和 后门 , 它们 都 被 锁上 了 。 但是 , 您 可以 突破 几个 窗口 。 你 检查 车库 门门 是 锁 着 的 , 进入 的 窗户 太高 了 , 够不着 。 也许 前台 区有 钥匙 。 \n > 你 问 他 发生 了 什么 事 \n “ 是 某种 奇怪 的 啮齿动物 ! ! 当 我 伸手 到 床 底下 时 它 咬 了 我 的 手 。 我 想 它 还 在 下面 ... ... ” 我 告诉 他 , 如果 他 准备 好 他 的 床 , 我会 抬起 床 撬棍 杀死 它 。 他 点头 同意 。'

In [18]:
m

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
writer = SummaryWriter('log/finetune')
n_iter = 0
train_loss = 100
val_loss = 100
for epoch in range(24):
    print(f"Epoch {epoch + 1}")
    pb = ProgressBar(len(train_recoard))
    pb.startjob()
    for x,m in data_generator_json(train_recoard):
    #for x in data_generator_content(texts[528:],sample_len=500,batch_size=2):
        n_iter += 1
        ret = gpt.train_step(
            (
                tf.constant(x[:,:-1]),
                tf.constant(x[:,1:])
            ),
            mask=m[:,1:],
        )
        writer.add_scalar("train/loss", ret["loss"].numpy(), n_iter)
        train_loss = ret["loss"].numpy()
        if n_iter % 10 == 0:
            valid_x,valid_m = valid_gen.__next__()
            ret = gpt.eval_step(
            (
                tf.constant(valid_x[:,:-1]),
                tf.constant(valid_x[:,1:])),
                mask=valid_m[:,1:],
            )
            writer.add_scalar("test/loss", ret["loss"].numpy(), n_iter)
            val_loss = ret["loss"].numpy()
        pb.info = f"tl: {train_loss} vl: {val_loss}"
        pb.complete(1)
    print()

Epoch 1
tl: 2.57421875 vl: 2.98046875 62.67 % [===============================>-------------------] 2820/4500 	 used:3118s eta:1857 ssss

/home/tiger/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


tl: 2.8125 vl: 2.603515625 100.00 % [==================================================>] 4500/4500 	 used:4960s eta:0 s2 ssssss
Epoch 2
tl: 2.591796875 vl: 2.60546875 35.93 % [=================>---------------------------------] 1617/4500 	 used:1772s eta:3159 ssssWarning: NaN or Inf found in input tensor.
tl: 2.873046875 vl: 3.166015625 100.00 % [==================================================>] 4500/4500 	 used:4921s eta:0 sssss
Epoch 3
tl: 2.71875 vl: 2.38671875 14.60 % [=======>-------------------------------------------] 657/4500 	 used:718s eta:4200 s01 sss

In [ ]:
gpt.save_weights('./gpt_weight_pretrain/tmp_weight')

In [ ]:
!ls -l ./gpt_weight_pretrain/tmp_weight*

# rewrite fp16 model (weight only)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
import json

In [2]:
tf.keras.backend.set_floatx('float16')
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0,
    train_size=499
)
gpt.load_weights('./gpt_weight_pretrain/tmp_weight')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16
        
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

def test_basic_logic():
    ids = cbpe.encode('今天天气还行')
    #print(ids)
    print("+" * 20)
    for i in range(10):
        output = gpt(tf.constant([ids]))
        #print(output[0].shape)
        nid = np.argmax(output[0, -1])
        ids += [nid]
        #print(i, cbpe.decode(ids))
        #print(np.argmax(output[0],axis=-1))
        print(cbpe.decode(np.argmax(output[0],axis=-1)))
        print('-' * 30)
test_basic_logic()
        
gpt.save_weights('./gpt_weight_pretrain/tmp_weight_test')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


float16 <dtype: 'float16'> True


Loading model cost 0.559 seconds.
Prefix dict has been built successfully.


++++++++++++++++++++
气 不错 不错行 
------------------------------
气 不错 不错行 ,
------------------------------
气 不错 不错行 , 
------------------------------
气 不错 不错行 , 咱
------------------------------
气 不错 不错行 , 咱们
------------------------------
气 不错 不错行 , 咱们 去
------------------------------
气 不错 不错行 , 咱们 去 
------------------------------
气 不错 不错行 , 咱们 去 郊
------------------------------
气 不错 不错行 , 咱们 去 郊外
------------------------------
气 不错 不错行 , 咱们 去 郊外 
------------------------------


In [4]:
!ls -l ./gpt_weight_pretrain/tmp_weight_test*

-rw-r--r-- 1 tiger tiger 5194222858 Jul 13 18:08 ./gpt_weight_pretrain/tmp_weight_test.data-00000-of-00001
-rw-r--r-- 1 tiger tiger      34188 Jul 13 18:08 ./gpt_weight_pretrain/tmp_weight_test.index


# save model (restart kernel here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
tf.keras.backend.set_floatx('float16')

In [2]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [3]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0
)
gpt.load_weights('./gpt_weight_pretrain/tmp_weight_test')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


#gpt.compile(
#    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
#    # Loss function to minimize
#    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#    # List of metrics to monitor
#)

float16 <dtype: 'float16'> True


In [4]:
ids = cbpe.encode('今天天气不错')

for i in range(10):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i, cbpe.decode(ids))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.561 seconds.
Prefix dict has been built successfully.


0 今天天气 不错 
1 今天天气 不错 ,
2 今天天气 不错 , 我
3 今天天气 不错 , 我 想
4 今天天气 不错 , 我 想 去
5 今天天气 不错 , 我 想 去 看看
6 今天天气 不错 , 我 想 去 看看 
7 今天天气 不错 , 我 想 去 看看 。
8 今天天气 不错 , 我 想 去 看看 。 
9 今天天气 不错 , 我 想 去 看看 。 ”


In [5]:
def batch_gather(a, b):
    return tf.gather(a, b, batch_dims=1)

def penalize_used(logits, output,penalize):
    # I want to change the indices of logits wherever the index is found in output
    change_tensor = tf.zeros_like(logits, dtype=logits.dtype)
    unique = tf.unique(output[0])[0]
    ones = tf.ones_like(unique, dtype=unique.dtype)
    indices = tf.expand_dims(unique, 1)
    updates = tf.scatter_nd(indices, ones, [logits.shape[1]])
    bool_tensor = tf.expand_dims(tf.cast(updates, tf.bool), 0)
    return tf.compat.v1.where(bool_tensor, logits * penalize, logits)

def top_p_sample(logits, inputs, num_samples=1, p=0.9,k=40,temperature=0.4,penalize=0.4):
    logits = penalize_used(logits,inputs,penalize)
    batch_size, vocab_size = logits.shape
    probs = tf.nn.softmax(logits / temperature, axis=-1)
    # [batch_size, vocab_perm]
    indices = tf.argsort(probs, direction='DESCENDING')
    cumulative_probabilities = tf.math.cumsum(batch_gather(probs, indices), axis=-1, exclusive=False)
    top_k_prob = cumulative_probabilities[0][k]

    # find the top pth index to cut off. careful we don't want to cutoff everything!
    # result will be [batch_size, vocab_perm]
    p_expanded = p if isinstance(p, float) else p#[:, None]
    exclude_mask = tf.logical_not(
        tf.logical_or(
            tf.logical_and(
                cumulative_probabilities < p_expanded, 
                cumulative_probabilities < top_k_prob
            ),
            tf.range(vocab_size)[None] < 1
        )
    )

    # OPTION A - sample in the sorted space, then unsort.
    logits_to_use = batch_gather(logits, indices) - tf.cast(exclude_mask, tf.float16) * 1e4
    sample_perm = tf.random.categorical(logits=logits_to_use, num_samples=num_samples)
    sample = batch_gather(indices, sample_perm)

    return tf.cast(sample, tf.int64)

In [6]:
@tf.function
def serve(inputs):
    return gpt(inputs, kv_cache=None, use_cache=True)


@tf.function
def serve_cache(inputs, kv_cache):
    return gpt(inputs, kv_cache=kv_cache, use_cache=True)

In [7]:
serve_concrete = serve.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp")
)

layer_size = 32
attention_head = 32
embedding_size = 2560

serve_cache_concrete = serve_cache.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
    tf.TensorSpec(shape=[
        layer_size, None, 2, attention_head,
        None, embedding_size // attention_head
    ], dtype=tf.float16, name="kv_cache")
)

In [8]:
r = serve_concrete(
    tf.constant([[1]], tf.int64)
)
r2 = serve_cache_concrete(
    tf.constant([[1]], tf.int64),
    r[1]
)

In [9]:
@tf.function
def sample(initial_inputs, length, p,k,temperature,penalize, num_samples=1):
    layer_size = 32
    embedding_size = 2560
    attention_head = 32

    i = tf.constant(0, dtype=tf.int64)
    initial_logits, kv_cache = serve(initial_inputs)
    inputs = top_p_sample(initial_logits[:, -1, :],initial_inputs, p=p,k=k,temperature=temperature,penalize=penalize)
    stores = tf.concat([initial_inputs, inputs], axis=1)

    def _cond(i, inputs, kv_cache, stores):
        return i < length

    def _body(i, inputs, kv_cache, stores):
        new_logits, new_kv_cache = serve_cache(inputs, kv_cache)
        
        new_inputs = top_p_sample(new_logits[:, -1, :],inputs, p=p,k=k,temperature=temperature,penalize=penalize)
        new_stores = tf.concat([stores, new_inputs], axis=-1)
        new_kv_cache = tf.concat([
            kv_cache,
            new_kv_cache
        ], axis=-2)
        new_i = i + 1
        return [new_i, new_inputs, new_kv_cache, new_stores]

    result = tf.while_loop(
        _cond, _body,
        loop_vars=[i, inputs, kv_cache, stores],
        shape_invariants=[
            tf.TensorShape(None),
            tf.TensorShape([None, None]),
            tf.TensorShape([
                layer_size, None, 2,
                attention_head, None,
                embedding_size // attention_head
            ]),
            tf.TensorShape([
                None, None
            ])
        ]
    )
    return result[-1]

In [10]:
gpt.save('./result_weights/tmp_weight', include_optimizer=False, signatures={
    'serving_default': sample.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
        tf.TensorSpec(shape=[None,], dtype=tf.int64, name="length"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="p"),
        tf.TensorSpec(shape=[], dtype=tf.int64, name="k"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="temperature"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="penalize")
    )
})

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./result_weights/tmp_weight/assets


In [11]:
ids = cbpe.encode('今天天气不错')

ret = sample(
    tf.constant([ids], dtype=tf.int64), # inp
    tf.constant(10, dtype=tf.int64), #length
    tf.constant(0.9, dtype=tf.float16), #p
    tf.constant(40, dtype=tf.int64), #k
    tf.constant(0.4, dtype=tf.float16), # temperature
    tf.constant(0.4, dtype=tf.float16) # penalize
)
print(ret)
print(cbpe.decode(ret.numpy().tolist()[0]))

tf.Tensor(
[[  837   259   497   788     8     9     8 29987    79    91 11851   133
      8    12     8]], shape=(1, 15), dtype=int64)
今天天气 不错 , 咱们 去 散步 吧 。 


In [12]:
!ls -l ./result_weights/tmp_weight

total 21660
drwxr-xr-x 2 tiger tiger     4096 Jul  5 15:21 assets
-rw-r--r-- 1 tiger tiger 22168232 Jul 13 18:11 saved_model.pb
drwxr-xr-x 3 tiger tiger     4096 Jul 13 18:11 variables


# read deploy model (you should restart kernel and go from here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorflow.keras.utils import multi_gpu_model
import random
from utils.gpt2_tokenizer import GPT2Tokenizer
import tensorflow_hub as hub
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [2]:
gpt = hub.load('./result_weights/tmp_weight/')

In [3]:
def sample_gpt(tokenizer, gpt, sentence, number=1, length=20, p=0.9,k=40,temperature=0.4,penalize=0.85):
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    p = tf.constant(p, dtype=tf.float16)
    k = tf.constant(k, dtype=tf.int64)
    temperature = tf.constant(temperature, dtype=tf.float16)
    penalize = tf.constant(penalize, dtype=tf.float16)
    ret = gpt.signatures['serving_default'](
        inp=inputs, 
        length=length, 
        p=p,
        k=k,
        temperature=temperature,
        penalize=penalize)['output_0']
    return [
        tokenizer.decode(s[len(inputs[0]):]).replace(' ', '')
        for s in ret.numpy()
    ]

In [4]:
q = f'''双十一'''
ret = sample_gpt(cbpe, gpt, q, 1, 500, p=0.9,k=16,temperature=1,penalize=0.75)
for x in ret:
    print(x)
    print('-' * 20)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.555 seconds.
Prefix dict has been built successfully.


,双十二,圣诞节等都可以做到的,现在很多品牌都可以做到,但真正做的好的却很少,这也是为什么代理商都不愿意做的原因,他们都想快速占领市场,占领更大的份额,但是却不愿意花大力气,也不愿意投入时间和人力,所以他们只能等待,等到他们认为可以了,他们就可以一统江湖了。代理商也不愿意放弃这个市场,因为只要他们不做,那么他们就会成为别人的笑柄,虽然这样的代价很大,但是却是个不错的选择。但是,最终,代理商还是失败了,因为没有人做,他们都高估了自己,而且他们都低估了事情的严重性,最终他们的代理权也被收回,因为他们已经没有足够的能力让其他人来做了。所以这就是为什么,他们只做了一年,就会失去全部的权力,甚至包括那些为他们提供支持的人。当他们失败时,人们会说:“他们只是昙花一现,他们只是运气不好,他们只是没有找到足够的人,所以他们的权力被收回了。”他们会说:“他们只做了几个月,他们只是运气不好,他们只是没有找到足够的人,所以他们的权力被收回了。”他们会说:“我不否认他们只做了几个月,他们只是运气不好,他们只是没有找到足够的人,所以他们的权力被收回了。”当他们失败时,人们会说:“他们只做了
--------------------


In [5]:
class Story():
    def __init__(self,beginning,story_max_len=200,context_len=12):
        self.story = [beginning]
        self.story_max_len = story_max_len
        self.context_len = context_len
        
    def response_quality_ok(self,response):
        if len(response) < 20:
            return False
        return True
    
    def action(self,action):
        action_str = "> 你" + action
        q = "".join(self.story[-self.context_len:]) + "\n" + action_str + "\n"
        q = q[-600:].replace("\n","")
        self.story.append("你" + action)
        response = ""
        for i in range(3):
            response = sample_gpt(cbpe, gpt, q, 1, 150,p=0.9,k=16,temperature=1,penalize=0.75)[0]
            response = '。'.join(response.split("。")[:-1]) + "。"
            if(self.response_quality_ok(response) == True):
                break
            else:
                print("response quality check failed,generating another one")
        
        #responsesp = response.split("。")
        #if(len(responsesp) > 2):
        #    response = ''.join(responsesp[:-1])
        #else:
        #    response = response.split(">")[0]
        #response = "。".join([:-1])
        #response = response.split(">")[0].strip()
        #response = "\n".join(response.split("\n")[:-1])
        self.story.append(response)
        
    def interactive(self):
        print("\n".join(self.story))
        while True:
            action = input("> 你")
            self.action(action)
            print(self.story[-1])

In [8]:
begin = f'''你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西'''
story = Story(begin)

In [9]:
story.interactive()

你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西


> 你 到附近的城镇


寻求帮助,他们会派警察到你的房子抓走你的父亲,但是他们没有带走他,你听到警察们对你说的第一句话,“你在干什么?”“我正在锯木头。”你皱着眉头说。“锯木头?”“是的,但我不是一个木匠。


> 你 找一个酒吧


response quality check failed,generating another one
侍者来给你的父亲锯木头,我知道这不太容易,但是我们可以。”“你真的认为他们会给我?”“嗯,”你说,“我们只是想做一下。”“好吧,如果你真的想要一些木料,我们可以为你找一个酒吧侍者。”“不,你是对的,但是我还是有一些木头可以做,锯木头并不是我的专长。


> 你 忽略这些人，找一个酒吧


的侍者,他可以帮你父亲修理。你走到他跟前,他正在修理一辆被打碎的拖车,他看起来很生气,你不知道他为什么生气,你也不知道是什么,因为你不确定自己是否真的知道。他说,“你把我们所有的木头都锯了吗?”“是的,他们不愿意给我锯木头,但我可以锯一些树枝,我需要一些木头。”“你在干什么?”“我在锯木头。


> 你 别锯木头了，去酒吧


喝一杯吧。我知道酒吧在哪儿。”“不,你别拿酒来了,我还是回家吧。


KeyboardInterrupt: Interrupted by user